In [ ]:
import pandas as pd
import os
import random
from datetime import timedelta
import numpy as np
from IPython.display import display


: 

# Conclusions

- orders_dataset is indeed the glue to retrive all information. order_id is a primary key in this dataset and there are no order_id in other dataset that does not appears in this dataset
- In 99.8% of the cases there's a 1:1 relationship between order_id and seller_id. For simplicity, we will remove the other 0.2% (though this can't be done in a real scenario)
- There are 5 payments type and a user may use more than one in an order. We will aggregate the payment at the type method and count the number of payment for each. Installments are only > 1 if the payment_type is credit card
- There are 8 order statuses but we can group them into 4: Delivered, shipped, cancelled and waiting for shipment
- Orders that appears with delivered status but don't appears in payment we will assume a payment done in voucher equal to the sum of items price + freight
- Orders that appears in the payments dataset but not in the items, will be treated as cancelled
- There are orders where payment != item_price + freight

# Utils
- Global paths
- functions

In [2]:
DATA_ROOT = "..\\data\\raw"
DATA_TRANSIENT = "..\\data\\staging"

In [3]:
def get_unique_val_col(df):
    """
    Return a dictionary with the names of the columns and the unique values in it
    """
    return {c:df[c].nunique() for c in df.columns}


def add_root(file, root):
    """
    Apend root to path
    """
    return os.path.join(root,file)


# Load raw data

In [4]:
orders =  pd.read_csv(add_root("olist_orders_dataset.csv", DATA_ROOT))

items = pd.read_csv(add_root("olist_order_items_dataset.csv", DATA_ROOT))

payments = pd.read_csv(add_root("olist_order_payments_dataset.csv", DATA_ROOT))

reviews = pd.read_csv(add_root("olist_order_reviews_dataset.csv", DATA_ROOT))


In [5]:
pd.to_datetime(orders.order_purchase_timestamp).min(), pd.to_datetime(orders.order_purchase_timestamp).max()

(Timestamp('2016-09-04 21:15:19'), Timestamp('2018-10-17 17:30:18'))

# Match Between order_id in orders and any table that contains order_id field

According to the data schema, the orders_dataset contains all order_id and it connects all the table. We expect that there is no order_id that appears in any of payments, reviews and items dataset that does not show in order table. We will prove this assumption but first, lets confirm that order_id is indeed a primary key for the orders dataset

In [6]:
(
                orders
                .agg(
                    unique = ("order_id", "nunique"),
                    count = ("order_id", "count")
                )
            )



,order_id
unique,99441
count,99441


Order_id in orders dataset is indeed a primary key. Now, the coverage with respect the items and payments confirms the expectation. There are 0 "right_only" values when joining orders dataset with other dataset on order_id.

In [7]:
print("counts of orders by source (both, left = order, right = payments)")

display(
    (
    orders
    .merge( payments.drop_duplicates(subset=["order_id"]), on="order_id", indicator=True)
    ._merge
    .value_counts()
)
)

print("counts of orders by source (both, left = order, right = items)")
display(
    (
    orders
    .merge( items.drop_duplicates(subset=["order_id"]), on="order_id", indicator=True)
    ._merge
    .value_counts()
)
)


print("counts of orders by source (both, left = order, right = reviews)")
display(
    (
    orders
    .merge( reviews.drop_duplicates(subset=["order_id"]), on="order_id", indicator=True)
    ._merge
    .value_counts()
)
)

counts of orders by source (both, left = order, right = payments)


both          99440
left_only         0
right_only        0
Name: _merge, dtype: int64

counts of orders by source (both, left = order, right = items)


both          98666
left_only         0
right_only        0
Name: _merge, dtype: int64

counts of orders by source (both, left = order, right = reviews)


both          98673
left_only         0
right_only        0
Name: _merge, dtype: int64

# Relationship seller - order_id


Relationship is not 1-1 strictly, but there are very few orders with two sellers, a 0.2%. For sake of simplicity, I'll remove those orders to enforce the 1-1 relationship so that the seller can be added to the order fact tables

In [8]:
orders_with_multiple_sellers = (
        items
        .query('order_item_id > 1')
        .groupby("order_id", as_index=False)
        .agg(
            n_seller = ('seller_id', 'nunique')
        )
        .query("n_seller > 1")
    )

print('number of total orders', items.order_id.nunique())
print('number of orders with at least two different sellers', orders_with_multiple_sellers.shape[0])



number of total orders 98666
number of orders with at least two different sellers 253


In [9]:
list_of_files_to_modify = ["olist_orders_dataset.csv", "olist_order_payments_dataset.csv", "olist_order_items_dataset.csv", "olist_order_reviews_dataset.csv"]


for file in list_of_files_to_modify:
    print(f"Modifying {file}")
    # filter out orders with multiple vendors
    (
        pd.read_csv(add_root(file, DATA_ROOT))
        .merge(orders_with_multiple_sellers, on='order_id', how='left', indicator=True)
        .query("_merge == 'left_only'")
        .drop(columns=['n_seller', '_merge'])
        .to_csv(add_root(file, DATA_TRANSIENT), index=False)
    )
    print("Complete\n")

Modifying olist_orders_dataset.csv
Complete

Modifying olist_order_payments_dataset.csv
Complete

Modifying olist_order_items_dataset.csv
Complete

Modifying olist_order_reviews_dataset.csv
Complete



# Load transient data

In [10]:
orders =  pd.read_csv(add_root("olist_orders_dataset.csv", DATA_TRANSIENT))

items = pd.read_csv(add_root("olist_order_items_dataset.csv", DATA_TRANSIENT))

payments = pd.read_csv(add_root("olist_order_payments_dataset.csv", DATA_TRANSIENT))

reviews = pd.read_csv(add_root("olist_order_reviews_dataset.csv", DATA_TRANSIENT))


# Payments option exploration

Goal is to review the number of different payment types: 
- What's the % of orders with more than payment type
- Do we need to store them all or just the one most used?
- Is there any other payment type that has more than 1 payment installments than credit card?

### Number of orders with multiple payment types


96.9% of orders only have one payment, this is the most common scenario. It's interesting to see that there are orders who have more than 5 payments even though the max number of payment types are 5, this means that we can make several payments of the same type in an order. 

Regarding the payments types, the names are mostly self-explanatory (boleto = cash). Voucher payment type will be considered as discount that the marketplace or seller offers to this users. The not_defined type seems it's strange given that the payment_value is 0 but a query into the main order table tells us that it's a cancelled one. We will keep the column into the curated table. 

In conclusion, for the curated table we will use the payment_type columns in combination of columns that indicates the number of payments for each type


In [11]:
# Number of orders by number of payments
(
    payments
    .groupby('order_id', as_index=False)
    .agg(
        n_payment = ("payment_sequential", "max")
    )
    .groupby("n_payment", as_index=False)
    .agg(
        n_order = ("order_id", "count")
    )
    .assign(
        perc_order = lambda df: df.n_order.cumsum().div(df.n_order.sum()) 
    )
)

,n_payment,n_order,perc_order
0,1,96157,0.969452
1,2,2452,0.994173
2,3,302,0.997217
3,4,107,0.998296
4,5,52,0.998820
5,6,36,0.999183
6,7,28,0.999466
7,8,11,0.999577
8,9,9,0.999667
9,10,5,0.999718


In [12]:
# Number of payments
(
    payments
    .payment_type
    .value_counts(normalize=True)
)

credit_card    0.739351
boleto         0.190448
voucher        0.055442
debit_card     0.014729
not_defined    0.000029
Name: payment_type, dtype: float64

In [13]:
# Orders with not_defined payment type
mask = payments.query("payment_type == 'not_defined'").order_id.iloc[0]

print("Orders with not_defined payment type")

display (
    (
    payments
    .query("order_id == @mask")
    .sort_values(by=["payment_sequential"])
)

)

print("Orders with not_defined payment type in the olist_orders_dataset.csv")

display (

    (
    orders
    .query("order_id == @mask")
)

)

Orders with not_defined payment type


,order_id,payment_sequential,payment_type,payment_installments,payment_value
51153,4637ca194b6387e2d538dc89b124b0ee,1,not_defined,1,0.0


Orders with not_defined payment type in the olist_orders_dataset.csv


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
39810,4637ca194b6387e2d538dc89b124b0ee,a73c1f73f5772cf801434bf984b0b1a7,canceled,2018-09-03 14:14:25,NaN,NaN,NaN,2018-09-10 00:00:00


Number of installments in exclusively of credit card payments. The cell outputs the % of orders with more than installment

In [14]:
# Payment type with more than one instalmments
(
    payments
    .query("payment_installments > 1")
    .payment_type
    .value_counts(normalize=True)
)

credit_card    1.0
Name: payment_type, dtype: float64

# Order statuses

98.% of orders have an status that's self-explanatory. The rest fall into other categories that I suspect are related to the states between a order is made and before it's being shiped. 

In [15]:
(
    orders
    [["order_id", "order_status"]]
    .order_status
    .value_counts(normalize=True)
    .cumsum()

)


delivered      0.970127
shipped        0.981288
canceled       0.987589
unavailable    0.993729
invoiced       0.996895
processing     0.999929
created        0.999980
approved       1.000000
Name: order_status, dtype: float64

The different timestamp fields and names helps us understand the other status except for the unavailable field. My guess is the following:
- Invoiced -> the order was created, approved and debited from the users and it's pending to be send to the carrier but this task has not started
- Processing -> the order was created, approved and debited from the users and it's pending to be send to the carrier and this task has started
- Created -> the order was just created, it hasn't been approved
- approved -> the order was created, approved and debited from the users but it hasn't been acknowledge the start of the task to send it to the carrier

The invoiced, processing, and created are quite similar. For simplicity, we'd group them into one single group called "waiting for shipment". The unavailable type requires more digging to understand what it is

In [16]:
print("Sample of order_status == invoiced")

display(
    (
    orders
    .query("order_status == 'invoiced'")
    .head(3)
)
)


print("Sample of order_status == processing")

display(
    (
    orders
    .query("order_status == 'processing'")
    .head(3)
)
)

print("Sample of order_status == created")
display(
    (
    orders
    .query("order_status == 'created'")
    .head(3)
)
)

print("Sample of order_status == approved")

display(
    (
    orders
    .query("order_status == 'approved'")
    .head(3)
)
)

print("Sample of order_status == unavailable")
display(
    (
    orders
    .query("order_status == 'unavailable'")
    .head(3)
)
)


print("Sample of order_status == canceled")
display(
    (
    orders
    .query("order_status == 'canceled'")
    .head(3)
)
)

Sample of order_status == invoiced


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaN,NaN,2017-05-09 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaN,NaN,2018-08-21 00:00:00
454,38b7efdf33dd5561f4f5d4f6e07b0414,021e84751ba0ead75b6d314a6ead88d9,invoiced,2017-08-01 18:17:41,2017-08-01 18:32:30,NaN,NaN,2017-08-28 00:00:00


Sample of order_status == processing


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaN,NaN,2017-10-03 00:00:00
323,d3c8851a6651eeff2f73b0e011ac45d0,957f8e082185574de25992dc659ebbc0,processing,2016-10-05 22:44:13,2016-10-06 15:51:05,NaN,NaN,2016-12-09 00:00:00
738,6a6c7d523fd59eb5bbefc007331af717,d954782ec6c0e911292c8a80757ef28d,processing,2017-11-24 20:09:33,2017-11-24 23:15:15,NaN,NaN,2017-12-20 00:00:00


Sample of order_status == created


...

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
44774,a2e4c44360b4a57bdff22f3a4630c173,8886130db0ea6e9e70ba0b03d7c0d286,approved,2017-02-06 20:18:17,2017-02-06 20:30:19,NaN,NaN,2017-03-01 00:00:00
88232,132f1e724165a07f6362532bfb97486e,b2191912d8ad6eac2e4dc3b6e1459515,approved,2017-04-25 01:25:34,2017-04-30 20:32:41,NaN,NaN,2017-05-22 00:00:00


Sample of order_status == unavailable


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaN,NaN,2017-12-05 00:00:00
585,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaN,NaN,2018-02-16 00:00:00
685,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaN,NaN,2017-09-05 00:00:00


Sample of order_status == canceled


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
396,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaN,NaN,2018-08-14 00:00:00
611,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaN,2018-02-22 00:00:00
1055,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,2018-01-26 21:42:18,NaN,2018-02-23 00:00:00


# Relationship between payments and items dataset

I'd like to understand the following:
- order_id match between the two: Does all orders in payment appears in items or viceversa?
- Does the sum of payment_value is equal to the sum of price + freight in items?

I'd add the order_status from the order dataset as will give more insights.

In [17]:

order_status = (
    orders
    [["order_id", "order_status"]]
)

item_gmv = (
    items
    .groupby("order_id", as_index=False)
    .agg(
        price_sum = ("price", "sum"),
        freight_sum = ("freight_value", "sum")
    )
    .assign(
        gmv = lambda df: df.price_sum + df.freight_sum
    )
)


payment_total = (
    payments
    .groupby("order_id", as_index=False)
    .agg(
        payment_sum = ("payment_value", "sum")
    )
)


payment_with_gmv = (
        payment_total
        .merge(item_gmv, on="order_id", how="outer", indicator=True)
        .merge(order_status, on="order_id", how="inner")
)


(
    payment_with_gmv
    .query("_merge != 'both'")
    .pivot_table(index="order_status", columns=["_merge"], values = ["order_id"], aggfunc="count")
)


order_id                
_merge       left_only right_only both
order_status                          
canceled           164          0    0
created              5          0    0
delivered            0          1    0
invoiced             2          0    0
shipped              1          0    0
unavailable        603          0    0

The majority of orders with payments and without items are unavailable and canceled. 

If we compare the number of unavailable without items (603)  with the total count by status (609), it looks like almost all unavailable orders have no items. An order is unavailable if the items are unavailable? If that's the case, this not optimal from the customer perspective as they payed for an item that's not available. Assuming the correct procedure is then to cancel the order and reverse the payment, we will consider order with unavailable status the same as canceled.

With respect to the orders with created, invoiced and shipped without items. I consider does not makes any sense, therefore, they will be treated as canceled too.

Finally, for the order that appears with a delivered status but no payment we will assume the payment was made using a voucher (platform/seller asume the cost)

In [18]:
display(
    (
    payment_with_gmv
    .query("_merge == 'right_only'")
)
)

mask =  (
    payment_with_gmv
    .query("_merge == 'right_only'")
    .order_id
    .iloc[0]
)


display(
    orders
    .query("order_id == @mask")
)


display(
      items
    .query("order_id == @mask")
)

,order_id,payment_sum,price_sum,freight_sum,gmv,_merge,order_status
99187,bfbd0f9bdef84302105ad712db648a6c,NaN,134.97,8.49,143.46,right_only,delivered


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
30625,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
83647,bfbd0f9bdef84302105ad712db648a6c,1,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83
83648,bfbd0f9bdef84302105ad712db648a6c,2,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83
83649,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19 23:11:33,44.99,2.83


Regarding the match between payment value and gmv (item price + freight) there are some orders where those the value does not match. This is an interesting case to note. 

In [19]:
(
    payment_with_gmv
    .query("_merge == 'both'")
    .assign(are_values_equal = lambda df: np.isclose(df.payment_sum, df.gmv, atol=0.1))
    .query("~are_values_equal")
    .assign(is_payment_higher_than_gmv =  lambda df: df.payment_sum > df.gmv)
    .is_payment_higher_than_gmv
    .value_counts()
)

True     239
False     18
Name: is_payment_higher_than_gmv, dtype: int64

# Seller creation date

Seller table does not have the date of when a seller signed up to the marketplace. The Olist closed deals does contains info about it but not for all merchants. For the sake of the project, I would like to produce a datamart which will store how sellers performs from the moment they signed up; it's essential to have the signup date. 

To solve this challenge, If the signup date is not available in the closed_deals, I'll use a random date between the date of the first order for a given seller in the orders dataset and the max_delta_to_randomize parameter 

In [20]:
def randomize_date_col(df, date_col, max_delta):
    """
    Randomize a date column by a random number defined by delta
    """
    df[f"{date_col}_random"] = df[date_col].apply(lambda x: x - timedelta(days=random.randint(1,max_delta)))
    return df
    

def coalesce_two_cols(df, cols, new_col=None):
    """
    Perform a SQL-like coalese function for two columns.
    """
    if not new_col:
        new_col = f'{cols[0]}_coalesce'

    col1, col2 = cols
    df[new_col] = np.where(df[col1].isnull(), df[col2], df[col1])
    return df

MAX_DELTA_TO_RANDOMIZE = 60

In [21]:
# get first order date for each merchant
orders = pd.read_csv(add_root("olist_orders_dataset.csv", DATA_TRANSIENT), 
                    parse_dates=["order_purchase_timestamp"],
                    usecols=["order_id", "order_purchase_timestamp"]
                    )\
                    .assign(purchase_date = lambda row: row["order_purchase_timestamp"].dt.date)\
                    .loc[:,["order_id", "purchase_date"]]
        

orders_sellers = pd.read_csv(add_root("olist_order_items_dataset.csv", DATA_TRANSIENT), usecols=["order_id", "seller_id"])\
            .drop_duplicates()


first_seller_order = orders_sellers\
            .merge(orders, how="left", on="order_id")\
            .groupby("seller_id", as_index=False)\
            .agg({"purchase_date":"min"})\
            .rename(columns={"purchase_date":"first_order_date"})\
            .pipe(randomize_date_col, "first_order_date", MAX_DELTA_TO_RANDOMIZE)\
            .drop(columns=["first_order_date"])

seller_closed_deals = pd.read_csv(add_root("olist_closed_deals_dataset.csv", DATA_ROOT), 
                                    parse_dates=["won_date"], 
                                    usecols=["seller_id", "won_date"]
                                    )\
                    .assign(deal_date = lambda row: row["won_date"].dt.date)\
                    .loc[:,["seller_id", "deal_date"]]


sellers_with_signupdate = pd.read_csv(add_root( "olist_sellers_dataset.csv", DATA_ROOT))\
                        .merge(first_seller_order, how="left", on="seller_id")\
                        .merge(seller_closed_deals, how="left", on="seller_id")\
                        .pipe(coalesce_two_cols, ["deal_date", "first_order_date_random"], "signup_date")\
                        .drop(columns=["first_order_date_random", "deal_date"])


sellers_with_signupdate.to_csv(add_root("olist_sellers_dataset.csv", DATA_TRANSIENT), index=False)